# Building a Model - Initial Runs

This notebook goes over how to create a data generator that reads in a section of a dataset and trains the model sequentially. It also goes through several different types of models. 

TensorFlow wasn't loading properly on my Windows computer, so I followed this link to create a new environment to run tensorflow things: https://medium.com/@mengjiunchiou/how-to-set-keras-with-tensorflow-with-conda-virtual-environment-on-ubuntu-5d56d22e3dc7

In [1]:
import matplotlib
matplotlib.use("Agg")
 
# import the necessary packages
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from skimage import io, filters, measure
from scipy import ndimage
from keras.models import Sequential
import cv2
from glob import glob
from sklearn.model_selection import train_test_split
from PIL import Image # for conversion to grayscale

C:\Users\echel\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:493: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\echel\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:494: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\echel\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:495: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\echel\AppData\Roaming\Python\Python36\site-

In [2]:
# Data have been pre-processed already. The raw images were converted
# to grayscale and were split into 25 smaller images. Labeled images
# were created from the original images and are a series of 0 (no 
# person) and 255 (a person). The labeled images were also split
# into 25 pieces so that they would match the original images. 

# Find location of image files and labeled images
data = glob('../data/raw/resized/with_people/splits/*hsv*.png')
labels = glob('../data/processed/dots/with_people/splits/*.png')

# Split into the training and testing data
train_X, test_X = train_test_split(data, test_size=0.25, random_state=33)
train_Y, test_Y = train_test_split(labels, test_size=0.25, random_state=33)

In [3]:
# Create function to make a data generator for the model
def gather_images(images, labels, batch_size=10): 
    """ Takes the original and labeled images, combines them into np """
    """ arrays, and passes to model"""
    while 1: 
        for offset in range(0, len(images), batch_size): 
            X = [] # empty list for training data
            Y = [] # empty list for labels 
            for img in images[offset:offset+batch_size]: # for each image in the list
                img_temp = cv2.imread(img)
                #img_temp = Image.open(img).convert('LA')
                #img_temp = cv2.cvtColor(img_temp, cv2.COLOR_BGR2HSV)
                img_flatten = np.array(img_temp) # create np array
                X.append(img_flatten) # and add to list for X
            for lab in labels[offset:offset+batch_size]: # for each label in the list
                label_temp = io.imread(lab, as_gray=True)
                labels_temp = measure.label(label_temp)
                label_flatten = labels_temp.max() # create np array
                Y.append(label_flatten) # and add to list for y
            yield (np.array(X), np.array(Y).reshape(len(Y),1)) # yield X and y for the model

# Attempt 1: Full Images - GrayScale
This uses full size images that have people in them, converted to grayscale. 

In [106]:
batch_size = 10

# Set up Convolutional Network
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(8, (2,2), padding="same",input_shape=(540, 960, 2), activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
  tf.keras.layers.Conv2D(16, (2,2), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
  tf.keras.layers.Conv2D(32, (2,2), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(4,4)),
  tf.keras.layers.Conv2D(64, (2,2), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(4,4)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(1),
])

model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['mean_squared_error', 'mean_absolute_error', 'accuracy'])

model.fit_generator(gather_images(train_X, train_Y, batch_size=batch_size),
                    steps_per_epoch = len(train_X)//batch_size, epochs=10)

Epoch 1/10
39/39 [==============================]39/39 [==============================] - 74s 2s/step - loss: 1848.7436 - mean_squared_error: 1848.7436 - mean_absolute_error: 21.0740 - acc: 0.0333

Epoch 2/10
39/39 [==============================]39/39 [==============================] - 69s 2s/step - loss: 28.3969 - mean_squared_error: 28.3969 - mean_absolute_error: 3.9576 - acc: 0.1128

Epoch 3/10
39/39 [==============================]39/39 [==============================] - 71s 2s/step - loss: 20.1725 - mean_squared_error: 20.1725 - mean_absolute_error: 3.0874 - acc: 0.1077

Epoch 4/10
39/39 [==============================]39/39 [==============================] - 70s 2s/step - loss: 16.9439 - mean_squared_error: 16.9439 - mean_absolute_error: 2.7732 - acc: 0.1590

Epoch 5/10
39/39 [==============================]39/39 [==============================] - 72s 2s/step - loss: 15.1568 - mean_squared_error: 15.1568 - mean_absolute_error: 2.5791 - acc: 0.1615

Epoch 6/10
39/39 [============

In [107]:
predictions = model.predict_generator(gather_images(test_X, test_Y, batch_size=batch_size), len(test_X)//batch_size)
print(predictions[0:10])

[[4.4597096]
 [4.0453725]
 [2.5865219]
 [1.876172 ]
 [2.7817848]
 [4.3727183]
 [3.9000027]
 [2.5050535]
 [3.3840609]
 [3.5734978]]


In [108]:
test_labels = next(gather_images(test_X, test_Y, batch_size=len(test_X)))[1]
print(test_labels[0:10])

[[7]
 [5]
 [1]
 [1]
 [2]
 [9]
 [3]
 [2]
 [3]
 [3]]


# Attempt 2: Full Images - HSV
This uses full size images that have people in them, converted to HSV.

In [110]:
batch_size = 10

# Set up Convolutional Network
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(8, (2,2), padding="same",input_shape=(540, 960, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
  tf.keras.layers.Conv2D(16, (2,2), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
  tf.keras.layers.Conv2D(32, (2,2), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(4,4)),
  tf.keras.layers.Conv2D(64, (2,2), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(4,4)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(1),
])

model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['mean_squared_error', 'mean_absolute_error', 'accuracy'])

model.fit_generator(gather_images(train_X, train_Y, batch_size=batch_size),
                    steps_per_epoch = len(train_X)//batch_size, epochs=10)

Epoch 1/10
39/39 [==============================]39/39 [==============================] - 87s 2s/step - loss: 2378.7755 - mean_squared_error: 2378.7755 - mean_absolute_error: 24.0337 - acc: 0.0385

Epoch 2/10
39/39 [==============================]39/39 [==============================] - 83s 2s/step - loss: 15.1942 - mean_squared_error: 15.1942 - mean_absolute_error: 2.4477 - acc: 0.1872

Epoch 3/10
39/39 [==============================]39/39 [==============================] - 84s 2s/step - loss: 11.7206 - mean_squared_error: 11.7206 - mean_absolute_error: 2.2168 - acc: 0.1897

Epoch 4/10
39/39 [==============================]39/39 [==============================] - 86s 2s/step - loss: 10.5186 - mean_squared_error: 10.5186 - mean_absolute_error: 2.1461 - acc: 0.2333

Epoch 5/10
39/39 [==============================]39/39 [==============================] - 90s 2s/step - loss: 7.3852 - mean_squared_error: 7.3852 - mean_absolute_error: 1.7826 - acc: 0.2410

Epoch 6/10
39/39 [==============

In [111]:
predictions = model.predict_generator(gather_images(test_X, test_Y, batch_size=batch_size), len(test_X)//batch_size)
print(predictions[0:10])

[[3.0930457]
 [2.5246828]
 [1.4346604]
 [1.3098843]
 [1.7063771]
 [1.902762 ]
 [3.913823 ]
 [1.0183108]
 [1.3632728]
 [3.1236968]]


In [112]:
test_labels = next(gather_images(test_X, test_Y, batch_size=len(test_X)))[1]
print(test_labels[0:10])

[[7]
 [5]
 [1]
 [1]
 [2]
 [9]
 [3]
 [2]
 [3]
 [3]]


# Attempt 3: Full Images - No Color Change
This uses full size images that have people in them.

In [114]:
batch_size = 10

# Set up Convolutional Network
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(8, (2,2), padding="same",input_shape=(540, 960, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
  tf.keras.layers.Conv2D(16, (2,2), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
  tf.keras.layers.Conv2D(32, (2,2), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(4,4)),
  tf.keras.layers.Conv2D(64, (2,2), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(4,4)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(1),
])

model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['mean_squared_error', 'mean_absolute_error', 'accuracy'])

model.fit_generator(gather_images(train_X, train_Y, batch_size=batch_size),
                    steps_per_epoch = len(train_X)//batch_size, epochs=10)

Epoch 1/10
39/39 [==============================]39/39 [==============================] - 85s 2s/step - loss: 1208.7269 - mean_squared_error: 1208.7269 - mean_absolute_error: 17.7038 - acc: 0.0231

Epoch 2/10
39/39 [==============================]39/39 [==============================] - 83s 2s/step - loss: 19.3600 - mean_squared_error: 19.3600 - mean_absolute_error: 3.0794 - acc: 0.1308

Epoch 3/10
39/39 [==============================]39/39 [==============================] - 80s 2s/step - loss: 14.3952 - mean_squared_error: 14.3952 - mean_absolute_error: 2.6225 - acc: 0.1462

Epoch 4/10
39/39 [==============================]39/39 [==============================] - 80s 2s/step - loss: 13.8770 - mean_squared_error: 13.8770 - mean_absolute_error: 2.4535 - acc: 0.1949

Epoch 5/10
39/39 [==============================]39/39 [==============================] - 81s 2s/step - loss: 11.1326 - mean_squared_error: 11.1326 - mean_absolute_error: 2.1599 - acc: 0.1923

Epoch 6/10
39/39 [============

In [115]:
predictions = model.predict_generator(gather_images(test_X, test_Y, batch_size=batch_size), len(test_X)//batch_size)
print(predictions[0:10])

[[1.3958495 ]
 [8.321123  ]
 [2.3532782 ]
 [1.035324  ]
 [0.70101446]
 [2.332126  ]
 [4.5625787 ]
 [0.87305564]
 [1.6255327 ]
 [2.5387604 ]]


In [116]:
test_labels = next(gather_images(test_X, test_Y, batch_size=len(test_X)))[1]
print(test_labels[0:10])

[[7]
 [5]
 [1]
 [1]
 [2]
 [9]
 [3]
 [2]
 [3]
 [3]]


# Attempt 4: Split Images - Grayscale
This uses split images (from full images with people in them) that have been converted to grayscale.

In [119]:
batch_size = 10

# Set up Convolutional Network
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(8, (2,2), padding="same",input_shape=(108, 192, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
  tf.keras.layers.Conv2D(16, (2,2), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
  tf.keras.layers.Conv2D(32, (2,2), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(4,4)),
  tf.keras.layers.Conv2D(64, (2,2), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(4,4)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(1),
])

model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['mean_squared_error', 'mean_absolute_error', 'accuracy'])

model.fit_generator(gather_images(train_X, train_Y, batch_size=batch_size),
                    steps_per_epoch = len(train_X)//batch_size, epochs=10)

Epoch 1/10
976/976 [==============================]976/976 [==============================] - 309s 316ms/step - loss: 3.6022 - mean_squared_error: 3.6022 - mean_absolute_error: 0.4892 - acc: 0.8026

Epoch 2/10
976/976 [==============================]976/976 [==============================] - 105s 107ms/step - loss: 0.2864 - mean_squared_error: 0.2864 - mean_absolute_error: 0.2316 - acc: 0.8929

Epoch 3/10
976/976 [==============================]976/976 [==============================] - 113s 116ms/step - loss: 0.2791 - mean_squared_error: 0.2791 - mean_absolute_error: 0.2264 - acc: 0.8944

Epoch 4/10
976/976 [==============================]976/976 [==============================] - 105s 108ms/step - loss: 0.2707 - mean_squared_error: 0.2707 - mean_absolute_error: 0.2226 - acc: 0.8972

Epoch 5/10
976/976 [==============================]976/976 [==============================] - 102s 105ms/step - loss: 0.2707 - mean_squared_error: 0.2707 - mean_absolute_error: 0.2240 - acc: 0.8969

Epoch

In [120]:
predictions = model.predict_generator(gather_images(test_X, test_Y, batch_size=batch_size), len(test_X)//batch_size)
print(predictions[0:10])

[[0.1062379 ]
 [0.12491779]
 [0.16854063]
 [0.10197994]
 [0.10313762]
 [0.11242331]
 [0.23321387]
 [0.10273636]
 [0.10166924]
 [0.10310157]]


In [121]:
test_labels = next(gather_images(test_X, test_Y, batch_size=len(test_X)))[1]
print(test_labels[0:10])

[[0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]]


# Attempt 5: Split Images - HSV
This uses split images (from full images with people in them) that have been converted to hsv.

In [142]:
batch_size = 30

# Set up Convolutional Network
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(8, (5,5), padding="same",input_shape=(108, 192, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(3,3)),
  tf.keras.layers.Conv2D(16, (5,5), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(3,3)),
  tf.keras.layers.Conv2D(32, (2,2), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
  tf.keras.layers.Conv2D(64, (2,2), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(1, activation='linear'),
])

model.compile(optimizer='adam',
              loss='mse',
              metrics=['mean_squared_error', 'mean_absolute_error'])

model.fit_generator(gather_images(train_X, train_Y, batch_size=batch_size),
                    steps_per_epoch = len(train_X)//batch_size, epochs=30)

Epoch 1/30
325/325 [==============================]325/325 [==============================] - 195s 601ms/step - loss: 1.8116 - mean_squared_error: 1.8116 - mean_absolute_error: 0.4146

Epoch 2/30
325/325 [==============================]325/325 [==============================] - 195s 599ms/step - loss: 0.2958 - mean_squared_error: 0.2958 - mean_absolute_error: 0.2440

Epoch 3/30
325/325 [==============================]325/325 [==============================] - 194s 598ms/step - loss: 0.2921 - mean_squared_error: 0.2921 - mean_absolute_error: 0.2450

Epoch 4/30
325/325 [==============================]325/325 [==============================] - 198s 610ms/step - loss: 0.2882 - mean_squared_error: 0.2882 - mean_absolute_error: 0.2465

Epoch 5/30
325/325 [==============================]325/325 [==============================] - 193s 593ms/step - loss: 0.2833 - mean_squared_error: 0.2833 - mean_absolute_error: 0.2473

Epoch 6/30
325/325 [==============================]325/325 [===============

In [ ]:
batch_size = 30

# Set up Convolutional Network
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(8, (7,7), padding="same",input_shape=(108, 192, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(3,3)),
  tf.keras.layers.Conv2D(16, (5,5), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(3,3)),
  tf.keras.layers.Conv2D(32, (2,2), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
  tf.keras.layers.Conv2D(64, (2,2), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(1, activation='linear'),
])

model.compile(optimizer='adam',
              loss='mse',
              metrics=['mean_squared_error', 'mean_absolute_error'])

model.fit_generator(gather_images(train_X, train_Y, batch_size=batch_size),
                    steps_per_epoch = len(train_X)//batch_size, epochs=30)

Epoch 1/30
325/325 [==============================]325/325 [==============================] - 321s 987ms/step - loss: 18.1632 - mean_squared_error: 18.1632 - mean_absolute_error: 0.8407

Epoch 2/30
325/325 [==============================]325/325 [==============================] - 294s 904ms/step - loss: 0.2967 - mean_squared_error: 0.2967 - mean_absolute_error: 0.2464

Epoch 3/30
325/325 [==============================]325/325 [==============================] - 301s 926ms/step - loss: 0.2871 - mean_squared_error: 0.2871 - mean_absolute_error: 0.2405

Epoch 4/30
325/325 [==============================]325/325 [==============================] - 293s 900ms/step - loss: 0.2797 - mean_squared_error: 0.2797 - mean_absolute_error: 0.2309

Epoch 5/30
325/325 [==============================]325/325 [==============================] - 291s 895ms/step - loss: 0.2662 - mean_squared_error: 0.2662 - mean_absolute_error: 0.2220

Epoch 6/30
325/325 [==============================]325/325 [=============

In [143]:
predictions = model.predict_generator(gather_images(test_X, test_Y, batch_size=batch_size), len(test_X)//batch_size)
print(predictions[0:10])

[[0.06639786]
 [0.597787  ]
 [0.05902122]
 [0.04196331]
 [0.04343889]
 [0.25613928]
 [0.04196331]
 [0.12044369]
 [0.04196331]
 [0.04298855]]


In [125]:
test_labels = next(gather_images(test_X, test_Y, batch_size=len(test_X)))[1]
print(test_labels[0:10])

[[0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]]


# Attempt 6: Split Images - No Color Change
This uses split images (from full images with people in them). 

In [127]:
batch_size = 10

# Set up Convolutional Network
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(8, (2,2), padding="same",input_shape=(108, 192, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
  tf.keras.layers.Conv2D(16, (2,2), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
  tf.keras.layers.Conv2D(32, (2,2), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(4,4)),
  tf.keras.layers.Conv2D(64, (2,2), padding="same", activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(4,4)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(1),
])

model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['mean_squared_error', 'mean_absolute_error', 'accuracy'])

model.fit_generator(gather_images(train_X, train_Y, batch_size=batch_size),
                    steps_per_epoch = len(train_X)//batch_size, epochs=10)

Epoch 1/10
976/976 [==============================]976/976 [==============================] - 102s 104ms/step - loss: 1.9541 - mean_squared_error: 1.9541 - mean_absolute_error: 0.4151 - acc: 0.8239

Epoch 2/10
976/976 [==============================]976/976 [==============================] - 97s 100ms/step - loss: 0.2725 - mean_squared_error: 0.2725 - mean_absolute_error: 0.2224 - acc: 0.8905

Epoch 3/10
976/976 [==============================]976/976 [==============================] - 105s 107ms/step - loss: 0.2613 - mean_squared_error: 0.2613 - mean_absolute_error: 0.2167 - acc: 0.8943

Epoch 4/10
976/976 [==============================]976/976 [==============================] - 104s 107ms/step - loss: 0.2624 - mean_squared_error: 0.2624 - mean_absolute_error: 0.2197 - acc: 0.8954

Epoch 5/10
976/976 [==============================]976/976 [==============================] - 102s 105ms/step - loss: 0.2544 - mean_squared_error: 0.2544 - mean_absolute_error: 0.2163 - acc: 0.8962

Epoch 

In [128]:
predictions = model.predict_generator(gather_images(test_X, test_Y, batch_size=batch_size), len(test_X)//batch_size)
print(predictions[0:10])

[[0.13789415]
 [0.25354084]
 [0.13789415]
 [0.13789415]
 [0.13789415]
 [0.1382464 ]
 [0.19708386]
 [0.1383607 ]
 [0.13789415]
 [0.13789415]]


In [129]:
test_labels = next(gather_images(test_X, test_Y, batch_size=len(test_X)))[1]
print(test_labels[0:10])

[[0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]]
